# Feature engineering

## Obtain the data

In [1]:
import os
import pandas as pd

In [2]:
# Earliest possible date is 2017-06-17
from_date = '2017-08-01'
until_date = '2017-09-01'

local_data_folder = 'data/raw' # do not end in /
output_folder = 'data/processed' # do not end in /

download_script = './download_data.sh'

dates = list(pd.date_range(from_date, until_date, freq='D').strftime('%Y-%m-%d'))

! mkdir -p {local_data_folder}

# We found it was more reliable to generate a bash script and run it, rather than
# run the commands in a python for-loop

with open(download_script, 'w') as f:
    f.write("#!/bin/bash\n")
    f.write("\nset -euo pipefail\n")
    f.write("\n# This script was generated to download data for multiple days\n")
    for date in dates:
        success_file =  os.path.join(local_data_folder, date, 'success')

        f.write("""
if [ ! -f {success_file} ]; then

    echo "Getting PDS dataset for date {date}"        
    mkdir -p {local_data_folder}/{date}
    aws s3 sync s3://deutsche-boerse-xetra-pds/{date} {local_data_folder}/{date} --no-sign-request
    touch {success_file}            
else
    echo "PDS dataset for date {date} already exists"
fi\n""".format(success_file=success_file, date=date, local_data_folder=local_data_folder))

        
! chmod +x {download_script}     
! head -n 15 {download_script} 

#!/bin/bash

set -euo pipefail

# This script was generated to download data for multiple days

if [ ! -f data/raw/2017-08-01/success ]; then

    echo "Getting PDS dataset for date 2017-08-01"        
    mkdir -p data/raw/2017-08-01
    aws s3 sync s3://deutsche-boerse-xetra-pds/2017-08-01 data/raw/2017-08-01 --no-sign-request
    touch data/raw/2017-08-01/success            
else
    echo "PDS dataset for date 2017-08-01 already exists"
fi


In [4]:
# execute the download script to retrieve the data
!  {download_script}

Getting PDS dataset for date 2017-08-01
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR00.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR00.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR02.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR02.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR05.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR05.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR01.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR01.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR03.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR03.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR04.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR04.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-08-01_BINS_XETR06.csv to data/raw/2017-08-01/2017-08-01_BINS_XETR06.csv
download: s3://deutsche-boerse-xetra-pds/2017-08-01/2017-

## Cleanse the data

In [45]:
import pandas as pd
import numpy as np
import glob, os
from datetime import datetime
import statsmodels.api as sm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
mpl.rcParams['figure.figsize'] = (15, 10) # use bigger graphs

In [42]:
def load_csv_dirs(data_dirs):
    files = []
    for data_dir in data_dirs:
        files.extend(glob.glob(os.path.join(data_dir, '*.csv')))
    return pd.concat(map(pd.read_csv, files), sort=False)

data_dir = local_data_folder + '/'
data_subdirs = map(lambda date: data_dir + date, dates)
unprocessed_df = load_csv_dirs(data_subdirs)
unprocessed_df.head(2)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A00XX9,P4N,POLYTEC HLDG AG INH. EO 1,Common stock,EUR,2504160,2017-08-01,14:00,15.180,15.180,15.180,15.18,90,1
1,CA0679011084,ABR,BARRICK GOLD CORP.,Common stock,EUR,2504196,2017-08-01,14:00,14.265,14.285,14.255,14.27,9832,27


In [9]:
unprocessed_df.shape

(1511548, 23)

In [6]:
unprocessed_df.info ()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1511548 entries, 0 to 8733
Data columns (total 14 columns):
ISIN              1511548 non-null object
Mnemonic          1511548 non-null object
SecurityDesc      1511548 non-null object
SecurityType      1511548 non-null object
Currency          1511548 non-null object
SecurityID        1511548 non-null object
Date              1511548 non-null object
Time              1511548 non-null object
StartPrice        1511548 non-null float64
MaxPrice          1511548 non-null float64
MinPrice          1511548 non-null float64
EndPrice          1511548 non-null float64
TradedVolume      1511548 non-null object
NumberOfTrades    1511548 non-null object
dtypes: float64(4), object(10)
memory usage: 173.0+ MB


In [11]:
unprocessed_df.count()

ISIN              2762277
Mnemonic          2762277
SecurityDesc      2762277
SecurityType      2762277
Currency          2762277
SecurityID        2762277
Date              2762277
Time              2762277
StartPrice        2762277
MaxPrice          2762277
MinPrice          2762277
EndPrice          2762277
TradedVolume      2762277
NumberOfTrades    2762277
dtype: int64

In [6]:
unprocessed_df.Mnemonic.value_counts()

EOAN    11876
BAS     11864
DAI     11852
SAP     11784
DBK     11756
FRE     11681
BMW     11650
TKA     11421
ALV     11387
DPW     11360
SIE     11230
LHA     11219
HEI     11163
CBK     11114
RWE     11101
FME     11064
DTE     11056
IFX     10991
ADS     10972
VOW3    10966
PSM     10942
BAYN    10675
CON     10629
MUV2    10611
VNA     10595
MRK     10594
BEI     10397
DB1     10146
LIN      9943
KGX      9873
        ...  
UIQS        1
X504        1
WOF         1
B500        1
WTD7        1
LYXS        1
KABN        1
LYPL        1
X0BR        1
X0D8        1
WTIM        1
IS31        1
LYYI        1
LYXM        1
12D         1
9GAH        1
OD7X        1
SRH         1
SDF1        1
D5BF        1
O4B         1
INP         1
PCFD        1
LOVG        1
SBU3        1
PCFC        1
SSN         1
WTED        1
VWSA        1
UIQ1        1
Name: Mnemonic, Length: 2116, dtype: int64

In [43]:
# we want the dates to be comparable to datetime.strptime()
unprocessed_df["CalcTime"] = pd.to_datetime("1900-01-01 " + unprocessed_df["Time"])
unprocessed_df["CalcDateTime"] = pd.to_datetime(unprocessed_df["Date"] + " " + unprocessed_df["Time"])
unprocessed_df.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
0,AT0000A00XX9,P4N,POLYTEC HLDG AG INH. EO 1,Common stock,EUR,2504160,2017-08-01,14:00,15.180,15.180,15.180,15.180,90,1,1900-01-01 14:00:00,2017-08-01 14:00:00
1,CA0679011084,ABR,BARRICK GOLD CORP.,Common stock,EUR,2504196,2017-08-01,14:00,14.265,14.285,14.255,14.270,9832,27,1900-01-01 14:00:00,2017-08-01 14:00:00
2,CA32076V1031,FMV,FIRST MAJESTIC SILVER,Common stock,EUR,2504197,2017-08-01,14:00,7.031,7.037,7.031,7.037,484,5,1900-01-01 14:00:00,2017-08-01 14:00:00
3,CH0012005267,NOT,"NOVARTIS NAM. SF 0,50",Common stock,EUR,2504217,2017-08-01,14:00,72.380,72.380,72.380,72.380,206,2,1900-01-01 14:00:00,2017-08-01 14:00:00
4,LU0274211480,DBXD,DB X-TRACK.DAX ETF(DR)1C,ETF,EUR,2504269,2017-08-01,14:00,119.220,119.220,119.220,119.220,218,1,1900-01-01 14:00:00,2017-08-01 14:00:00


In [46]:
# Filter common stock
# Filter between trading hours 08:00 and 20:00
# Exclude auctions (those are with TradeVolume == 0)
only_common_stock = unprocessed_df[unprocessed_df.SecurityType == 'Common stock']
time_fmt = "%H:%M"
opening_hours_str = "08:00"
closing_hours_str = "20:00"
opening_hours = datetime.strptime(opening_hours_str, time_fmt)
closing_hours = datetime.strptime(closing_hours_str, time_fmt)

cleaned_common_stock = only_common_stock[(only_common_stock.TradedVolume > 0) & \
                  (only_common_stock.CalcTime >= opening_hours) & \
                  (only_common_stock.CalcTime <= closing_hours)]
cleaned_common_stock.head(2)

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
0,AT0000A00XX9,P4N,POLYTEC HLDG AG INH. EO 1,Common stock,EUR,2504160,2017-08-01,14:00,15.180,15.180,15.180,15.18,90,1,1900-01-01 14:00:00,2017-08-01 14:00:00
1,CA0679011084,ABR,BARRICK GOLD CORP.,Common stock,EUR,2504196,2017-08-01,14:00,14.265,14.285,14.255,14.27,9832,27,1900-01-01 14:00:00,2017-08-01 14:00:00


In [47]:
bymnemonic = cleaned_common_stock[['Mnemonic', 'TradedVolume']].groupby(['Mnemonic']).sum()
number_of_stocks = 100
top = bymnemonic.sort_values(['TradedVolume'], ascending=[0]).head(number_of_stocks)
top.head(10)

,TradedVolume
Mnemonic,
DBK,186411896
EOAN,179359412
SNH,148249208
CBK,129310817
DTE,95852480
LHA,50780846
IFX,50058533
RWE,50051833
HDD,49120269


In [48]:
top_k_stocks = list(top.index.values)
cleaned_common_stock = cleaned_common_stock[cleaned_common_stock.Mnemonic.isin(top_k_stocks)]
cleaned_common_stock.head()

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,CalcTime,CalcDateTime
1,CA0679011084,ABR,BARRICK GOLD CORP.,Common stock,EUR,2504196,2017-08-01,14:00,14.265,14.285,14.255,14.270,9832,27,1900-01-01 14:00:00,2017-08-01 14:00:00
6,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2017-08-01,14:00,11.430,11.430,11.430,11.430,459,3,1900-01-01 14:00:00,2017-08-01 14:00:00
8,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN AG INH,Common stock,EUR,2504314,2017-08-01,14:00,33.975,33.975,33.925,33.945,2119,22,1900-01-01 14:00:00,2017-08-01 14:00:00
12,DE000A0LD2U1,AOX,ALSTRIA OFFICE REIT-AG,Common stock,EUR,2504379,2017-08-01,14:00,12.360,12.370,12.355,12.360,7085,22,1900-01-01 14:00:00,2017-08-01 14:00:00
15,DE000A0WMPJ6,AIXA,AIXTRON SE NA O.N.,Common stock,EUR,2504428,2017-08-01,14:00,7.511,7.511,7.499,7.499,4889,5,1900-01-01 14:00:00,2017-08-01 14:00:00


In [13]:
sorted_by_index = cleaned_common_stock.set_index(['Mnemonic', 'CalcDateTime']).sort_index()
sorted_by_index.head()

ISIN       SecurityDesc  SecurityType  \
Mnemonic CalcDateTime                                                         
1COV     2017-08-01 08:00:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-08-01 08:02:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-08-01 08:03:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-08-01 08:05:00  DE0006062144  COVESTRO AG  O.N.  Common stock   
         2017-08-01 08:06:00  DE0006062144  COVESTRO AG  O.N.  Common stock   

                             Currency SecurityID        Date   Time  \
Mnemonic CalcDateTime                                                 
1COV     2017-08-01 08:00:00      EUR    2505008  2017-08-01  08:00   
         2017-08-01 08:02:00      EUR    2505008  2017-08-01  08:02   
         2017-08-01 08:03:00      EUR    2505008  2017-08-01  08:03   
         2017-08-01 08:05:00      EUR    2505008  2017-08-01  08:05   
         2017-08-01 08:06:00      EUR    2505008  2017-08-01  08:06   

                              StartPrice  MaxPrice  MinPrice     ...      \
Mnemonic CalcDateTime                                            ...       
1COV     2017-08-01 08:00:00       66.07     66.13     66.03     ...       
         2017-08-01 08:02:00       66.08     66.13     66.08     ...       
         2017-08-01 08:03:00       66.15     66.20     66.15     ...       
         2017-08-01 08:05:00       66.14     66.14     66.14     ...       
         2017-08-01 08:06:00       66.13     66.17     66.13     ...       

                              TradedVolume NumberOfTrades            CalcTime  \
Mnemonic CalcDateTime                                                           
1COV     2017-08-01 08:00:00           883             11 1900-01-01 08:00:00   
         2017-08-01 08:02:00           360              4 1900-01-01 08:02:00   
         2017-08-01 08:03:00           408              6 1900-01-01 08:03:00   
         2017-08-01 08:05:00          5145             11 1900-01-01 08:05:00   
         2017-08-01 08:06:00           975              6 1900-01-01 08:06:00   

                             HourOfDay  MinOfHour  MinOfDay  DayOfWeek  \
Mnemonic CalcDateTime                                                    
1COV     2017-08-01 08:00:00         8          0       480          1   
         2017-08-01 08:02:00         8          2       482          1   
         2017-08-01 08:03:00         8          3       483          1   
         2017-08-01 08:05:00         8          5       485          1   
         2017-08-01 08:06:00         8          6       486          1   

                              DayOfYear  MonthOfYear  WeekOfYear  
Mnemonic CalcDateTime                                             
1COV     2017-08-01 08:00:00        213            8          31  
         2017-08-01 08:02:00        213            8          31  
         2017-08-01 08:03:00        213            8          31  
         2017-08-01 08:05:00        213            8          31  
         2017-08-01 08:06:00        213            8          31  

[5 rows x 21 columns]

In [49]:
non_empty_days = sorted(list(cleaned_common_stock['Date'].unique()))
len(non_empty_days), non_empty_days[0:2], '...', non_empty_days[-3:-1]

(24, ['2017-08-01', '2017-08-02'], '...', ['2017-08-30', '2017-08-31'])

In [50]:
print ("Ideal data count for any stock: {}".format (44*12*60))
print ("Observation count per mnemonic:")
cleaned_common_stock.Mnemonic.value_counts()

Ideal data count for any stock: 31680
Observation count per mnemonic:


EOAN    10458
BAS     10431
DAI     10418
SAP     10352
DBK     10328
FRE     10266
BMW     10226
TKA     10053
ALV      9960
DPW      9941
LHA      9832
SIE      9807
HEI      9801
CBK      9722
RWE      9721
FME      9675
DTE      9641
PSM      9606
ADS      9576
IFX      9576
VOW3     9565
BAYN     9285
MRK      9269
VNA      9265
CON      9245
MUV2     9222
BEI      9117
DB1      8972
KGX      8714
LIN      8679
        ...  
ADV      4687
DEZ      4418
GYC      4321
PBB      4100
TEG      4048
SANT     3934
BVB      3689
P1Z      3669
ZIL2     3455
AOX      3306
MDG1     3142
BPE5     2888
TTI      2821
NOA3     2582
GAZ      2557
MLP      2410
ABR      2385
CAP      2201
TINA     2162
AB1      2115
QSC      1924
SWVK     1844
WCMK     1594
PA8      1584
VODI     1568
AT1      1300
AD1       690
ANO       278
LLD       184
SVAB       54
Name: Mnemonic, Length: 100, dtype: int64

In [51]:
import datetime
def build_index(non_empty_days, from_time, to_time):
    date_ranges = []
    for date in non_empty_days:
        yyyy, mm, dd = date.split('-')
        from_hour, from_min = from_time.split(':')
        to_hour, to_min = to_time.split(':')    
        t1 = datetime.datetime(int(yyyy), int(mm), int(dd), int(from_hour),int(from_min),0)
        t2 = datetime.datetime(int(yyyy), int(mm), int(dd), int(to_hour),int(to_min),0) 
        date_ranges.append(pd.DataFrame({"OrganizedDateTime": pd.date_range(t1, t2, freq='1Min').values}))
    agg = pd.concat(date_ranges, axis=0) 
    agg.index = agg["OrganizedDateTime"]
    return agg
new_datetime_index = build_index(non_empty_days, opening_hours_str, closing_hours_str)["OrganizedDateTime"].values
new_datetime_index

array(['2017-08-01T08:00:00.000000000', '2017-08-01T08:01:00.000000000',
       '2017-08-01T08:02:00.000000000', ...,
       '2017-09-01T19:58:00.000000000', '2017-09-01T19:59:00.000000000',
       '2017-09-01T20:00:00.000000000'], dtype='datetime64[ns]')

In [57]:
def basic_stock_features(input_df, mnemonic, new_time_index):
    stock = sorted_by_index.loc[mnemonic].copy()
    
    stock = stock.reindex(new_time_index)
    
    features = ['MinPrice', 'MaxPrice', 'EndPrice', 'StartPrice']
    for f in features:
        stock[f] = stock[f].fillna(method='ffill')   
    
    features = ['TradedVolume', 'NumberOfTrades']
    for f in features:
        stock[f] = stock[f].fillna(0.0)
        
    stock['HourOfDay'] = stock.index.hour
    stock['MinOfHour'] = stock.index.minute
    stock['MinOfDay'] = stock.index.hour*60 + stock.index.minute

    stock['DayOfWeek'] = stock.index.dayofweek
    stock['DayOfYear'] = stock.index.dayofyear
    stock['MonthOfYear'] = stock.index.month
    stock['WeekOfYear'] = stock.index.weekofyear
    
    stock['Mnemonic'] = mnemonic
    unwanted_features = ['ISIN', 'SecurityDesc', 'SecurityType', 'Currency', 'SecurityID', 'Date', 'Time', 'CalcTime']
    return stock.drop (unwanted_features, axis=1)

In [38]:
s = sorted_by_index.loc['EVD'].copy ()

In [40]:
s.index.dayofyear

Int64Index([213, 213, 213, 213, 213, 213, 213, 213, 213, 213,
            ...
            244, 244, 244, 244, 244, 244, 244, 244, 244, 244],
           dtype='int64', name='CalcDateTime', length=5363)

In [58]:
stocks = []
for stock in top_k_stocks:
    stock = basic_stock_features(sorted_by_index, stock, new_datetime_index)
    stocks.append(stock)
# prepared should contain the numeric features for all top k stocks,
# for all days in the interval, for which there were trades (that means excluding weekends and holidays)
# for all minutes from 08:00 until 20:00
# in minutes without trades the prices from the last available minute are carried forward
# trades are filled with zero for such minutes
# a new column called HasTrade is introduced to denote the presence of trades
prepared = pd.concat(stocks, axis=0)

In [59]:
prepared.Mnemonic = prepared.Mnemonic.astype('category')
prepared.Mnemonic.value_counts()

ZIL2    17304
EVK     17304
DAI     17304
DB1     17304
DBK     17304
DEZ     17304
DLG     17304
DPW     17304
DTE     17304
DWNI    17304
EOAN    17304
EVD     17304
EVT     17304
ZAL     17304
FME     17304
FNTN    17304
FRE     17304
G1A     17304
GAZ     17304
GYC     17304
HDD     17304
HEI     17304
HEN3    17304
IFX     17304
CON     17304
CEC     17304
CBK     17304
CAP     17304
AB1     17304
ABR     17304
        ...  
VOW3    17304
WAF     17304
WCMK    17304
WDI     17304
SHA     17304
SDF     17304
LHA     17304
SAZ     17304
LIN     17304
LLD     17304
LXS     17304
MDG1    17304
MLP     17304
MRK     17304
MUV2    17304
NDX1    17304
NOA3    17304
O2D     17304
P1Z     17304
PA8     17304
PAH3    17304
PBB     17304
PSM     17304
QIA     17304
QSC     17304
RKET    17304
RWE     17304
SANT    17304
SAP     17304
1COV    17304
Name: Mnemonic, Length: 100, dtype: int64

**TODO** Convert timestamp to more meaningful derived features

**TODO** Integrate with plotly for histograms, correlation matrices, etc

In [60]:
sorted_by_index.loc['EVD'].tail ()

,ISIN,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,...,TradedVolume,NumberOfTrades,CalcTime,HourOfDay,MinOfHour,MinOfDay,DayOfWeek,DayOfYear,MonthOfYear,WeekOfYear
CalcDateTime,,,,,,,,,,,,,,,,,,,,,
2017-09-01 15:26:00,DE0005470306,CTS EVENTIM KGAA,Common stock,EUR,2504934,2017-09-01,15:26,37.920,37.920,37.900,...,1701,23,1900-01-01 15:26:00,15,26,926,4,244,9,35
2017-09-01 15:27:00,DE0005470306,CTS EVENTIM KGAA,Common stock,EUR,2504934,2017-09-01,15:27,37.900,37.910,37.900,...,852,8,1900-01-01 15:27:00,15,27,927,4,244,9,35
2017-09-01 15:28:00,DE0005470306,CTS EVENTIM KGAA,Common stock,EUR,2504934,2017-09-01,15:28,37.900,37.900,37.875,...,507,10,1900-01-01 15:28:00,15,28,928,4,244,9,35
2017-09-01 15:29:00,DE0005470306,CTS EVENTIM KGAA,Common stock,EUR,2504934,2017-09-01,15:29,37.865,37.875,37.840,...,599,15,1900-01-01 15:29:00,15,29,929,4,244,9,35
2017-09-01 15:30:00,DE0005470306,CTS EVENTIM KGAA,Common stock,EUR,2504934,2017-09-01,15:30,37.925,37.930,37.925,...,125,3,1900-01-01 15:30:00,15,30,930,4,244,9,35


In [61]:
prepared.tail()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,HourOfDay,MinOfHour,MinOfDay,DayOfWeek,DayOfYear,MonthOfYear,WeekOfYear,Mnemonic
CalcDateTime,,,,,,,,,,,,,,
2017-09-01 19:56:00,37.925,37.93,37.925,37.93,0.0,0.0,19,56,1196,4,244,9,35,EVD
2017-09-01 19:57:00,37.925,37.93,37.925,37.93,0.0,0.0,19,57,1197,4,244,9,35,EVD
2017-09-01 19:58:00,37.925,37.93,37.925,37.93,0.0,0.0,19,58,1198,4,244,9,35,EVD
2017-09-01 19:59:00,37.925,37.93,37.925,37.93,0.0,0.0,19,59,1199,4,244,9,35,EVD
2017-09-01 20:00:00,37.925,37.93,37.925,37.93,0.0,0.0,20,0,1200,4,244,9,35,EVD


In [23]:
prepared.head()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,HourOfDay,MinOfHour,MinOfDay,DayOfWeek,DayOfYear,MonthOfYear,WeekOfYear,Mnemonic
CalcDateTime,,,,,,,,,,,,,,
2017-08-01 08:00:00,15.130,15.135,15.125,15.130,25775.0,29.0,8.0,0.0,480.0,1.0,213.0,8.0,31.0,DBK
2017-08-01 08:01:00,15.125,15.135,15.115,15.130,25212.0,20.0,8.0,1.0,481.0,1.0,213.0,8.0,31.0,DBK
2017-08-01 08:02:00,15.130,15.155,15.130,15.150,156268.0,107.0,8.0,2.0,482.0,1.0,213.0,8.0,31.0,DBK
2017-08-01 08:03:00,15.145,15.155,15.140,15.150,67265.0,43.0,8.0,3.0,483.0,1.0,213.0,8.0,31.0,DBK
2017-08-01 08:04:00,15.150,15.155,15.140,15.155,20731.0,20.0,8.0,4.0,484.0,1.0,213.0,8.0,31.0,DBK


In [62]:
# We save both in csv in pickle. Generally we'd read from the pickeled format because 
# it preserves the indices, but for cases where pkl cannot be read, we also output a csv format
! mkdir -p {output_folder}
prepared.to_csv(output_folder + '/cooked_v3.csv')

prepared.to_pickle(output_folder + '/cooked_v3.pkl')

In [26]:
!ls -lh {output_folder}

total 385M
-rw-r--r-- 1 825712516 1896053708 149M Dec 10 21:34 cooked_v3.csv
-rw-r--r-- 1 825712516 1896053708 213M Dec 10 21:34 cooked_v3.pkl
